### Environment

In [1]:
import openai
import os
import json
import torch
import numpy as np
import pandas as pd

In [2]:
os.environ["OPENAI_API_KEY"] = "sk-hjI1pBX4AYr79tBs9f55Ce345f9041A3B2D8Fa0c665c42Ac"
os.environ["OPENAI_API_BASE"] = "https://ai-yyds.com/v1"

### RAG

##### Loading

In [3]:
from langchain_community.document_loaders import PyPDFLoader

In [4]:
loader = PyPDFLoader("/gemini/data-1/初赛训练数据集.pdf")
data=loader.load()

In [5]:
data_list=[]
for i in range (0,len(data)):
  page=data[i].metadata["page"]
  data[i].metadata["page"]=page+1
  if i>=2 and i<=6:
    data[i].metadata["category"]="catalog"
  else:
    data[i].metadata["category"]="content"
  if data[i].page_content!="":
    data_list.append(data[i])

我现在估计需要进行一下这边的预处理的工作。
这边基本上都是没有加上任何的meta的。估计之后需要加上meta.
注意这边的meta信息是不对的，因为这边的meta信息page是从0开始的，但是page的信息需要从1开始。所以这边有这样的一个page的更改。



这边我估计就是直接放弃所有的split的过程

##### Indexing

In [6]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [7]:
from langchain.vectorstores import Chroma
vectordb = Chroma.from_documents(
    documents=data_list,
    embedding=embeddings)

In [8]:
vectordb._collection.count()

334

#### 构建的retriever_k

In [9]:
retriever_k =vectordb.as_retriever(search_kwargs={"k": 5})

#### Cross encoding

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

/root/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
model = AutoModelForSequenceClassification.from_pretrained("/gemini/pretrain")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("/gemini/pretrain")

In [37]:
def create_documents(query):
    results=retriever_k.get_relevant_documents(query)
    doc_strings = [doc.page_content for doc in results]
    doc_pages=[doc.metadata["page"] for doc in results]
    doc_s=list(np.unique(doc_strings))
    doc_p=list(np.unique(doc_pages))
    queries=[query]*len(doc_s)
    features_k = tokenizer(queries ,doc_s,  padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        score_k = model(**features_k).logits

    scores=score_k.cpu().numpy()
    df=pd.DataFrame([doc_s,doc_p,scores]).T
    df.columns=["text","page","score"]
    df_sorted=df.sort_values(by='score', ascending=False)
    first_three_element = df_sorted.loc[:3,"text"]
    first_three_pages=df_sorted.loc[:3,"page"]
    return first_three_element,first_three_pages

#### retrieval qa with cross encoding

In [13]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0,
    model_name="gpt-3.5-turbo-1106",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    openai_api_base=os.environ["OPENAI_API_BASE"],
)

/root/miniconda3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [32]:
from langchain_core.prompts import ChatPromptTemplate
qa_system_prompt = """你是一个汽车方面的专家，请结合给定的资料，并回答最终的问题。请如实回答，如果问题在资料中找不到答案，请回答不知道。
资料：{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        ("human", "{question}"),
    ]
 )

In [33]:
from langchain_core.runnables import RunnablePassthrough
chain = ({"context":RunnablePassthrough()  , "question": RunnablePassthrough()}
    | qa_prompt
    | llm
 )

#### Testing

In [16]:
questions = json.load(open("/gemini/data-1/questions.json"))

In [38]:
for i in range(len(questions)):
  query=questions[i]["question"]
  first_three_element,first_three_pages=create_documents(query)
  pages=["page_"+str(p) for p in first_three_pages]
  questions[i]["reference"]=",".join(pages)
  chunks=[]
  for chunk in chain.stream({"context":"\n\n".join(first_three_element),"question":query}):
    chunks.append(chunk.content)
  result="".join(chunks)
  questions[i]['answer'] = result

PermissionDeniedError: Error code: 403 - {'error': {'message': 'user quota is not enough (request id: 20240219074301945488035S97Jx5jX)', 'type': 'one_api_error', 'param': '', 'code': 'insufficient_user_quota'}}

In [36]:
# Serializing json
json_object = json.dumps(questions, indent=4)

# Writing to sample.json
with open("sample_openai_rerank_3.json", "w") as outfile:
    outfile.write(json_object)

有点明白为什么之前就是使用一个doc来做的原因

In [35]:
questions

[{'question': '“前排座椅通风”的相关内容在第几页？',
  'answer': '“前排座椅通风”的相关内容在第251页。',
  'reference': 'page_116'},
 {'question': '"关于车辆的儿童安全座椅固定装置，在哪一页可以找到相关内容？"',
  'answer': '根据提供的资料，关于车辆的儿童安全座椅固定装置的相关内容可以在第123页找到。',
  'reference': 'page_118'},
 {'question': '“打开前机舱盖”的相关信息在第几页？',
  'answer': '抱歉，我无法回答你的问题。',
  'reference': 'page_116'},
 {'question': '“打开前机舱盖”这个操作在哪一页？',
  'answer': '“打开前机舱盖”这个操作在给定的资料中并没有明确的页码信息。因此，根据提供的资料，无法准确回答这个问题。',
  'reference': 'page_114'},
 {'question': '“查看行车记录仪视频”这一项内容在第几页？',
  'answer': '抱歉，根据提供的资料，没有找到关于“查看行车记录仪视频”这一项内容在第几页的信息。因此无法回答这个问题。',
  'reference': 'page_79'},
 {'question': '请问Lynk&Co领克汽车的事件数据记录系统（EDR）主要记录哪些信息？',
  'answer': '抱歉，根据提供的资料，我无法找到关于Lynk&Co领克汽车的事件数据记录系统（EDR）主要记录哪些信息的答案。建议您直接联系Lynk&Co领克汽车销售有限公司或者领克中心，以获取准确的信息。',
  'reference': 'page_1'},
 {'question': '问题：事件数据记录系统（EDR）中的数据是否可以被黑客利用进行恶意攻击？',
  'answer': '根据提供的资料，事件数据记录系统（EDR）用于记录车辆发生碰撞事故的相关信息，如车辆的行驶速度和制动状态。这些数据可以帮助相关人员了解事件发生时车辆的状态，便于Lynk&Co领克依照相关法律要求和其他规定行事。另外，在符合适用法律法规的前提下，这些信息还可用于工程研究，有助于Lynk &Co领克持续提升

对于这边这个发现有很多的“因此无法回答这个问题。”

这边应该可以这种情况，后续做处理。

我怀疑这边不行的主要原因，应该是在于split上面的问题。然后是对应的embedding上面的问题。embedding如果是large的应该还可以。